<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# %load_ext autoreload 
# %autoreload 2

import sys
import json
import re
import pandas as pd
import pickle
from numpy.testing import assert_allclose
from pathlib import Path

from dask.distributed import Client
from sklearn import set_config
from macro_utils import load_datastore
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns

# internal
sys.path.append(r"/home/osindi/ami")
sys.path.append(r"/home/osindi/Symawofo")
sys.path.append(r"C:\SyMAWoFlo\Symawofo")
sys.path.append(r"C:\ami")



In [2]:
from symawofo import FeatureFrame, DataSplitter, WindowFrame, get_client
client = get_client(0)


In [3]:
from ami.run.macrobot import MacrobotRunner

asset = 'US_Equity_Fut'
runner = MacrobotRunner(asset)


In [ ]:
runner.get_source_into_next_cell()

In [5]:
from ami.build.loss import get_loss_fn
from ami.macrobot.jobspec import Jobspec
from symawofo.ensemble import BaggingRegressor, FeatureEnsemble
from symawofo.transformers import (Clipper, Difference, Lag, make_column_transformer,
                                   make_pipeline,
                                   StandardScaler, TransformedTargetRegressor)
from symawofo import FeatureFrame, DataSplitter, WindowFrame
from symawofo.signals.wf import ThresholdSignSignal
from symawofo.pnl import PNL, AssetType
from macro_utils import DotDict
from ami.build.common import (make_pca_column_transformer, make_pca_diff_column_transformer,
    make_local_data_passthrough)
from ami.macrobot.models import GRURegressor, CNNRegressor
from ami.build.loss import SampleWeightDecay


#%%
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

#%% [markdown]
## Feature transformation and Embeded models


#%% 

def make_preprocessing(target, prediction_lag):
    target_prepipeline = make_pipeline(
            Difference(),
            Lag(-prediction_lag)
    )
    return target_prepipeline.fit_transform(target)


def make_feature_transform(config: DotDict):
    tr_pca_levels = make_pca_column_transformer(config.PCA)
    tr_pca_diffs = make_pca_diff_column_transformer(config.PCA_diff)
    tr_level_features = [('passthrough', config.pass_through)]
    tr_diffs = [(Difference(1), config.diff)]
    tr_local = make_local_data_passthrough(config)

    ct = tr_pca_levels + tr_pca_diffs + tr_level_features + tr_diffs + tr_local

    return make_pipeline(
            make_column_transformer(*ct, remainder='drop'),
            StandardScaler(),
            Clipper(-config.clip_level, config.clip_level)
    )


def make_model(**kwargs):
    kwargs['loss'] = get_loss_fn(kwargs['loss'])
    sample_weights_fn = SampleWeightDecay(kwargs.get('decay')) if kwargs.get('decay') else None
    if kwargs.pop('type') == 'GruModel':
        return GRURegressor(_sample_weight_fn=sample_weights_fn, **kwargs)
    else:
        return CNNRegressor(_sample_weight_fn=sample_weights_fn, **kwargs)


def get_feature_callable(features, group=False, ignore_pca=True):
    def feature_callable(X):
        results = []
        pca_features = x.filter_by_ancestors_names(list(x.features.query(f"transformer_name=='PCA'").index))
    
        for f in features:
            temp = X.filter_by_ancestors_names(f)
        
            if ignore_pca and '_PCA' not in f:
                temp = [i for i in temp if i not in pca_features]
        
            if len(temp)==1 or group=='separate':
                results += temp
            else:
                assert group, f"Feature had multiple children, choose from following:\n "+','.join(temp)
                if group=='combine':
                    results.append(tuple(temp))
                else:
                    assert False, "group should be False, 'combined' or 'separate'"
        results_ex_dups = [*dict.fromkeys(results)]
        results_keys = [str(i) if isinstance(i, tuple) else i for i in results_ex_dups]
        results_dict = dict(zip(results_keys, results_ex_dups))
        return results_dict
    return feature_callable


def get_FeatureEnsemble_params(spec):
    testing_features = spec.pop('features')
    features_callable = get_feature_callable(testing_features, group=spec.pop('group'), ignore_pca=spec.pop('ignore_pca'))
    spec[spec.pop('type')] = features_callable
    return spec


def make_ensemble_model(jobspec: DotDict):
    model = make_model(**jobspec.model)
    feature_transform = make_feature_transform(jobspec.features_transform)

    if ensemble := dict(jobspec.ensemble):
        if ensemble.pop('type') == 'bagging':
            model = BaggingRegressor(model, **ensemble)

    if feature_testing := jobspec.feature_testing:
        feature_params = get_FeatureEnsemble_params(dict(feature_testing))
        model = FeatureEnsemble(model, **feature_params)

    estimator = make_pipeline(
        feature_transform,
        model
    )
    target_postpipline = make_pipeline(
        StandardScaler(with_mean=jobspec.target_transform.with_mean),
        Clipper(-jobspec.target.clip_level, jobspec.target.clip_level))
    return TransformedTargetRegressor(estimator, target_postpipline)


def make_data_splitter(jobspec: DotDict):
    return  DataSplitter(
            **jobspec.data_splitter,
            drop_remaining=True,
            last_training_window=True
    )


def make_window_frame(jobspec: DotDict, data):
    target = data[jobspec.target.name]
    modeling_features = FeatureFrame(data=data)
    modeling_target = make_preprocessing(target, jobspec.target.prediction_lag)
    splitter = make_data_splitter(jobspec)
    targetRegressor = make_ensemble_model(jobspec)
    wf = WindowFrame(
            features=modeling_features,
            target=modeling_target,
            splitter=splitter,
            estimator=targetRegressor,
            parallel=True
    )
    return wf

def make_signal(jobspec: DotDict, window_frame:WindowFrame):
    return ThresholdSignSignal(window_frame, **jobspec.signal)

def make_pnl(jobspec: DotDict, signal):
    return PNL(signal,
        asset_type=AssetType(log=True, level=False),
        **jobspec.trade)

#%% [markdown]
## Main execution flow


def main():
    # %% [markdown]
    ###  prepare the job spec and fetch the dta

    # %%
    from ami.run.macrobot import MacrobotRunner
    spec = Jobspec({
    "data_splitter": {
        "expanding": 1,
        "initial_training_size": 375,
        "test_size": 13
    },
    "feature_testing": None,
#         {
#         "strategy": 'drop_one',
#         "group": 'separate', #False, separate or combine
#         "type": 'core_features', #varying_features or core_features
#         "features": [
#             "eq_ctry2_UnitedStates50_20",
#             "US_AAII_Net_StandardScaler_Clipper",
#             "US_Equity_Vol"
#             ]
#         },
    "ensemble": {
        "agg_fn": None,
        "max_samples": 0.9,
        "n_estimators": 1,
        "type": "bagging"
    },
    "features_transform": {
        "PCA": [
            "US_ConsConf"
        ],
        "PCA_diff": [
            "US_Growth",
            "US_ConsConf",
            "US_Inflation",
            "US_Credit_Growth",
            "US_Inflexp",
            "US_FCI",
            "US_Risk",
            "US_WageExp"
        ],
        "clip_level": 8,
        "diff": [
            "GLOB_Equity",
            "US_Breakeven55",
            "US_Bond_10y",
            "US_erp_est",
            "US_AAII_Net",
            "US_CECI",
            "US_eps_upgrade_ratio"
        ],
        "pass_through": [
            "eq_per_flows_USA",
            "eq_ctry2_UnitedStates50_20",
            "US_AAII_Net",
            "US_CFTC_SPX",
            "US_erp_est",
            "US_GSInn",
            "US_Fiscal_Bal",
            "GLOB_CB_Bal",
            "US_CB_Bal",
            "US_2y_v_OIS",
            "US_ForYieldSp",
            "GLOB_EM_HCSp",
            "US_Buybacks_SPX_BBG",
            "US_MA_BBG",
            "GLOB_Equity_Skew",
            "US_Equity_Fut_Chg13",
            "US_Equity_Fut_Chg4",
            "US_CECI",
            "US_eps_upgrade_ratio",
            "ES_Volume_MA",
            "US_Equity_Fut_MASignal",
            "US_Equity_Fut_RSI",
            "US_EqBond_Corr_Ch",
            "US_EqBond_Corr",
            "US_PE",
            "US_Equity_Feature_2",
            "US_Equity_Vol"
        ]
    },
    "model": {
        "activation": "selu",
        "batch_size": 1000,
        "batchnorm": True,
        "batchnorm_trainable": True,
        "batchnorm_training": False,
        "decay": 0.3,
        "dropout": 0.05,
        "epochs": 100,
        "lookback_periods": 13,
        "loss": "mse",
        "nodes": 15,
        "optimizer": "adam",
        "recurrent_dropout": 0.2,
        "regularizer_l1": 0.003,
        "regularizer_l2": 0.0,
        "type": "GruModel",
        "use_bias": False
    },
    "signal": {
        "c": 0.1,
        "threshold": None
    },
    "target": {
        "clip_level": 3,
        "name": "US_Equity_Fut",
        "prediction_lag": 1
    },
    "target_transform": {
        "type": "StandardScaler",
        "with_mean": False
    },
    "trade": {
        "t_cost": 0.0001
    }
})


    runner = globals().get('runner', MacrobotRunner(spec))
    data = runner.get_data()

    # %% [markdown]
    ###  build the model

    # %%
    wf = make_window_frame(spec, data)
    signal = make_signal(spec, wf)
    pnl = make_pnl(spec, signal)
#     pnl.get_pnl()

    # %%
    return pnl


In [6]:
pnl = main()
pnl.get_pnl()

Monitor progress at http://127.0.0.1:8787/status
Monitor progress at http://127.0.0.1:8787/status


2012-12-07    1.000000
2012-12-14    0.999194
2012-12-21    1.011151
2012-12-28    0.982127
2013-01-04    1.036860
                ...   
2020-10-23    1.534805
2020-10-30    1.590475
2020-11-06    1.520739
2020-11-13    1.497797
2020-11-20    1.505574
Name: US_Equity_Fut_Difference1_Lead1_pred_0, Length: 416, dtype: float64

In [8]:
x, y = pnl.signal.wf[0].get_samples(return_2d=True, return_featureframe=True)

In [25]:
def feature_callable2(X):
    results = []
    pca_features = x.filter_by_ancestors_names(list(x.features.query(f"transformer_name=='PCA'").index))
    
    for f in features:
        temp = X.filter_by_ancestors_names(f)
        
        if ignore_pca and '_PCA' not in f:
            temp = [i for i in temp if i not in pca_features]
            
        if len(temp)==1 or group=='separate':
            results += temp
        else:
            assert group, f"Feature had multiple children, choose from following:\n "+','.join(temp)
            if group=='combine':
                results.append(tuple(temp))
            else:
                assert False, "group should be False, 'combined' or 'separate'"
    results_ex_dups = [*dict.fromkeys(results)]
    results_keys = [str(i) if isinstance(i, tuple) else i for i in results_ex_dups]
    print(results_keys)
    assert False
    results_dict = dict(zip(results_keys, results_ex_dups))
    return results_dict

In [28]:
x.features

shortname    dtype  \
name                                                                   
US_ConsConf_PCA1_StandardScaler_Clipper            Clipper1  float64   
Diff_US_ConsConf_PCA1_StandardScaler_Clipper       Clipper2  float64   
Diff_US_Credit_Growth_PCA1_StandardScaler_Clipper  Clipper3  float64   
Diff_US_FCI_PCA1_StandardScaler_Clipper            Clipper4  float64   
Diff_US_Growth_PCA1_StandardScaler_Clipper         Clipper5  float64   
...                                                     ...      ...   
US_Bond_10y                                            None  float64   
US_erp_est                                             None  float64   
US_AAII_Net                                            None  float64   
US_CECI                                                None  float64   
US_eps_upgrade_ratio                                   None  float64   

                                                          transformer_uid  \
name                                                                        
US_ConsConf_PCA1_StandardScaler_Clipper            i87A3SfK9d5BWKmbmh6r6H   
Diff_US_ConsConf_PCA1_StandardScaler_Clipper       i87A3SfK9d5BWKmbmh6r6H   
Diff_US_Credit_Growth_PCA1_StandardScaler_Clipper  i87A3SfK9d5BWKmbmh6r6H   
Diff_US_FCI_PCA1_StandardScaler_Clipper            i87A3SfK9d5BWKmbmh6r6H   
Diff_US_Growth_PCA1_StandardScaler_Clipper         i87A3SfK9d5BWKmbmh6r6H   
...                                                                   ...   
US_Bond_10y                                                          None   
US_erp_est                                                           None   
US_AAII_Net                                                          None   
US_CECI                                                              None   
US_eps_upgrade_ratio                                                 None   

                                                  transformer_name  \
name                                                                 
US_ConsConf_PCA1_StandardScaler_Clipper                    Clipper   
Diff_US_ConsConf_PCA1_StandardScaler_Clipper               Clipper   
Diff_US_Credit_Growth_PCA1_StandardScaler_Clipper          Clipper   
Diff_US_FCI_PCA1_StandardScaler_Clipper                    Clipper   
Diff_US_Growth_PCA1_StandardScaler_Clipper                 Clipper   
...                                                            ...   
US_Bond_10y                                                   None   
US_erp_est                                                    None   
US_AAII_Net                                                   None   
US_CECI                                                       None   
US_eps_upgrade_ratio                                          None   

                                                                                  transformer_params  \
name                                                                                                   
US_ConsConf_PCA1_StandardScaler_Clipper            {'_get_columns_override': None, '_name': 'Clip...   
Diff_US_ConsConf_PCA1_StandardScaler_Clipper       {'_get_columns_override': None, '_name': 'Clip...   
Diff_US_Credit_Growth_PCA1_StandardScaler_Clipper  {'_get_columns_override': None, '_name': 'Clip...   
Diff_US_FCI_PCA1_StandardScaler_Clipper            {'_get_columns_override': None, '_name': 'Clip...   
Diff_US_Growth_PCA1_StandardScaler_Clipper         {'_get_columns_override': None, '_name': 'Clip...   
...                                                                                              ...   
US_Bond_10y                                                                                     None   
US_erp_est                                                                                      None   
US_AAII_Net                                                                                     None   
US_CECI                                                 

In [26]:
x.filter_by_ancestors_names(list(x.features.query(f"transformer_name=='PCA'").index))

['US_ConsConf_PCA1_StandardScaler_Clipper',
 'Diff_US_ConsConf_PCA1_StandardScaler_Clipper',
 'Diff_US_Credit_Growth_PCA1_StandardScaler_Clipper',
 'Diff_US_FCI_PCA1_StandardScaler_Clipper',
 'Diff_US_Growth_PCA1_StandardScaler_Clipper',
 'Diff_US_Inflation_PCA1_StandardScaler_Clipper',
 'Diff_US_Inflexp_PCA1_StandardScaler_Clipper',
 'Diff_US_Risk_PCA1_StandardScaler_Clipper',
 'Diff_US_WageExp_PCA1_StandardScaler_Clipper']

In [27]:
ignore_pca = True
feature_testing = {
        "strategy": 'drop_one',
        "group": 'separate', #False, separate or combine
        "type": 'varying_features', #varying_features or core_features
        "features": [
            "eq_ctry2_UnitedStates50_20",
            "US_AAII_Net_StandardScaler_Clipper",
            "US_Equity_Vol"
            ]
        }
features = feature_testing['features']
group = feature_testing['group']
feature_callable2(x)

['eq_ctry2_UnitedStates50_20_StandardScaler_Clipper', 'US_AAII_Net_StandardScaler_Clipper', 'US_Equity_Vol_StandardScaler_Clipper']


AssertionError: 

In [13]:
x.features.query(f"transformer_name=='PCA'").index

Index(['US_ConsConf_PCA1', 'Diff_US_ConsConf_PCA1',
       'Diff_US_Credit_Growth_PCA1', 'Diff_US_FCI_PCA1', 'Diff_US_Growth_PCA1',
       'Diff_US_Inflation_PCA1', 'Diff_US_Inflexp_PCA1', 'Diff_US_Risk_PCA1',
       'Diff_US_WageExp_PCA1'],
      dtype='object', name='name')

In [ ]:
from ami.macrobot.charting_tools import *

In [ ]:
avg_pnl = chart_fe_averages(pnl.signal, runner.jobspec)

In [ ]:
fe_summary(pnl, avg_pnl)

In [ ]:
widgets.interactive(plot_each_feature, pnl=widgets.fixed(pnl), averaged_pnl=widgets.fixed(avg_pnl), feature_group=list(avg_pnl.columns))    